In [118]:
#!/usr/bin/env python3

import re
import urllib.request
import urllib.parse
import http.cookiejar
import pandas as pd
from lxml.html import fragment_fromstring
from collections import OrderedDict
from decimal import Decimal

def get_data(*args, **kwargs):
    url = 'http://www.fundamentus.com.br/resultado.php'
    cookie_jar = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
    opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201'),
                         ('Accept', 'text/html, text/plain, text/css, text/sgml, */*;q=0.01')]

    # Aqui estão os parâmetros de busca das ações
    # Estão em branco para que retorne todas as disponíveis
    data = {'pl_min': '',
            'pl_max': '',
            'pvp_min': '',
            'pvp_max' : '',
            'psr_min': '',
            'psr_max': '',
            'divy_min': '',
            'divy_max': '',
            'pativos_min': '',
            'pativos_max': '',
            'pcapgiro_min': '',
            'pcapgiro_max': '',
            'pebit_min': '',
            'pebit_max': '',
            'fgrah_min': '',
            'fgrah_max': '',
            'firma_ebit_min': '',
            'firma_ebit_max': '',
            'margemebit_min': '',
            'margemebit_max': '',
            'margemliq_min': '',
            'margemliq_max': '',
            'liqcorr_min': '',
            'liqcorr_max': '',
            'roic_min': '',
            'roic_max': '',
            'roe_min': '',
            'roe_max': '',
            'liq_min': '',
            'liq_max': '',
            'patrim_min': '',
            'patrim_max': '',
            'divbruta_min': '',
            'divbruta_max': '',
            'tx_cresc_rec_min': '',
            'tx_cresc_rec_max': '',
            'setor': '',
            'negociada': 'ON',
            'ordem': '1',
            'x': '28',
            'y': '16'}

    with opener.open(url, urllib.parse.urlencode(data).encode('UTF-8')) as link:
        content = link.read().decode('ISO-8859-1')

    pattern = re.compile('<table id="resultado".*</table>', re.DOTALL)
    content = re.findall(pattern, content)[0]
    page = fragment_fromstring(content)
    result = OrderedDict()

    for rows in page.xpath('tbody')[0].findall("tr"):
        result.update({rows.getchildren()[0][0].getchildren()[0].text: {'Cotacao': todecimal(rows.getchildren()[1].text),
                                                                        'P/L': todecimal(rows.getchildren()[2].text),
                                                                        'P/VP': todecimal(rows.getchildren()[3].text),
                                                                        'PSR': todecimal(rows.getchildren()[4].text),
                                                                        'DY': todecimal(rows.getchildren()[5].text),
                                                                        'P/Ativo': todecimal(rows.getchildren()[6].text),
                                                                        'P/Cap.Giro': todecimal(rows.getchildren()[7].text),
                                                                        'P/EBIT': todecimal(rows.getchildren()[8].text),
                                                                        'P/ACL': todecimal(rows.getchildren()[9].text),
                                                                        'EV/EBIT': todecimal(rows.getchildren()[10].text),
                                                                        'EV/EBITDA': todecimal(rows.getchildren()[11].text),
                                                                        'Mrg.Ebit': todecimal(rows.getchildren()[12].text),
                                                                        'Mrg.Liq.': todecimal(rows.getchildren()[13].text),
                                                                        'Liq.Corr.': todecimal(rows.getchildren()[14].text),
                                                                        'ROIC': todecimal(rows.getchildren()[15].text),
                                                                        'ROE': todecimal(rows.getchildren()[16].text),
                                                                        'Liq.2meses': todecimal(rows.getchildren()[17].text),
                                                                        'Pat.Liq': todecimal(rows.getchildren()[18].text),
                                                                        'Div.Brut/Pat.': todecimal(rows.getchildren()[19].text),
                                                                        'Cresc.5anos': todecimal(rows.getchildren()[20].text)}})
    
    return result
    
def todecimal(string):
  string = string.replace('.', '')
  string = string.replace(',', '.')

  if (string.endswith('%')):
    string = string[:-1]
    return Decimal(string) / 100
  else:
    return Decimal(string)

if __name__ == '__main__':
    from waitingbar import WaitingBar
    
    progress_bar = WaitingBar('[*] Downloading...')
    result = get_data()
    progress_bar.stop()

    result_format = '{0:<7} {1:<7} {2:<10} {3:<7} {4:<10} {5:<7} {6:<10} {7:<10} {8:<10} {9:<11} {10:<11} {11:<7} {12:<11} {13:<11} {14:<7} {15:<11} {16:<5} {17:<7}'
    print(result_format.format('Papel',
                               'Cotacao',
                               'P/L',
                               'P/VP',
                               'PSR',
                               'DY',
                               'P/Ativo',
                               'P/Cap.Giro',
                               'P/EBIT',
                               'P/ACL',
                               'EV/EBIT',
                               'EV/EBITDA',
                               'Mrg.Ebit',
                               'Mrg.Liq.',
                               'Liq.Corr.',
                               'ROIC',
                               'ROE',
                               'Liq.2meses',
                               'Pat.Liq',
                               'Div.Brut/Pat.',
                               'Cresc.5anos'))

    print('-' * 190)

    for key, value in result.items():
        print(result_format.format(key,
                                   value['Cotacao'],
                                   value['P/L'],
                                   value['P/VP'],
                                   value['PSR'],
                                   value['DY'],
                                   value['P/Ativo'],
                                   value['P/Cap.Giro'],
                                   value['P/EBIT'],
                                   value['P/ACL'],
                                   value['EV/EBIT'],
                                   value['EV/EBITDA'],
                                   value['Mrg.Ebit'],
                                   value['Mrg.Liq.'],
                                   value['Liq.Corr.'],
                                   value['ROIC'],
                                   value['ROE'],
                                   value['Liq.2meses'],
                                   value['Pat.Liq'],
                                   value['Div.Brut/Pat.'],
                                   value['Cresc.5anos']))


8.68    0.5058      0.3084      3.99    0.0563      0.0458 12643000.00
GOAU4   10.92   22.45      1.02    0.299      0.0165  0.187      0.91       4.67       -1.22       9.52        4.96    0.064       0.0362      2.42    0.0492      0.0456 144543000.00
CIEL3   3.67    22.76      1.09    0.931      0.0177  0.127      1.57       10.53      -2.55       14.15       6.35    0.0884      0.0409      1.11    0.058       0.0479 145358000.00
GRND3   7.75    22.82      1.90    4.052      0.0216  1.737      2.93       23.61      3.04        19.10       14.25   0.1716      0.1775      9.74    0.1171      0.0835 13299200.00
QUAL3   31.79   22.95      5.09    4.443      0.0092  2.326      10.12      11.60      -16.19      12.06       8.03    0.383       0.1944      2.36    0.3187      0.2217 104130000.00
SBSP3   40.60   23.15      1.28    1.576      0.0339  0.581      90.25      6.44       -1.29       9.56        6.54    0.2446      0.0681      1.07    0.0944      0.0553 169636000.00
BPAC5   24.45  

In [121]:
print (result.items())

al('0.0311')}), ('PGMN3', {'Cotacao': Decimal('10.62'), 'P/L': Decimal('72.68'), 'P/VP': Decimal('2.57'), 'PSR': Decimal('0.705'), 'DY': Decimal('0.00'), 'P/Ativo': Decimal('0.831'), 'P/Cap.Giro': Decimal('3.90'), 'P/EBIT': Decimal('14.23'), 'P/ACL': Decimal('-6.16'), 'EV/EBIT': Decimal('15.13'), 'EV/EBITDA': Decimal('8.59'), 'Mrg.Ebit': Decimal('0.0495'), 'Mrg.Liq.': Decimal('0.0097'), 'Liq.Corr.': Decimal('1.65'), 'ROIC': Decimal('0.0847'), 'ROE': Decimal('0.0354'), 'Liq.2meses': Decimal('6010950.00'), 'Pat.Liq': Decimal('1830370000.00'), 'Div.Brut/Pat.': Decimal('0.52'), 'Cresc.5anos': Decimal('0.0421')}), ('HAPV3', {'Cotacao': Decimal('17.90'), 'P/L': Decimal('73.81'), 'P/VP': Decimal('8.48'), 'PSR': Decimal('8.243'), 'DY': Decimal('0.0033'), 'P/Ativo': Decimal('5.056'), 'P/Cap.Giro': Decimal('43.05'), 'P/EBIT': Decimal('48.96'), 'P/ACL': Decimal('-35.41'), 'EV/EBIT': Decimal('48.59'), 'EV/EBITDA': Decimal('33.64'), 'Mrg.Ebit': Decimal('0.1684'), 'Mrg.Liq.': Decimal('0.1118'), 'Liq

In [120]:
list = []
for key, value in result.items():
    list.append(result_format.format(key,
                            value['Cotacao'],
                            value['P/L'],
                            value['P/VP'],
                            value['PSR'],
                            value['DY'],
                            value['P/Ativo'],
                            value['P/Cap.Giro'],
                            value['P/EBIT'],
                            value['P/ACL'],
                            value['EV/EBIT'],
                            value['EV/EBITDA'],
                            value['Mrg.Ebit'],
                            value['Mrg.Liq.'],
                            value['Liq.Corr.'],
                            value['ROIC'],
                            value['ROE'],
                            value['Liq.2meses'],
                            value['Pat.Liq'],
                            value['Div.Brut/Pat.'],
                            value['Cresc.5anos']))
